In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import shutil
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import time
from torchvision.models import resnet18
from torchvision.models import mobilenet_v2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Download and Load data

In [ ]:
BATCH_SIZE = 45
transform = transforms.Compose(
    [#transforms.Resize((224,224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5])
     ]
)


transform_224 = transforms.Compose(
    [transforms.Resize((224,224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5])
     ]
)
#Test
testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)




testset_224 = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform_224)
testloader_224 = torch.utils.data.DataLoader(testset_224, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)




100%|██████████| 169001437/169001437 [00:05<00:00, 28529667.17it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


#Code for testing model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

criterion = nn.CrossEntropyLoss()

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item())

def valid_model(model, validloader, device):
    model.eval()
    # results = {'loss':[],
    #            'accuracy':[]}
    loss_list= []
    acc_list = []


    for i, data in enumerate(validloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs =model(inputs)

        loss = criterion(outputs, labels)
        acc = accuracy(outputs,labels)


        loss_list.append(loss.item())
        acc_list.append(acc.item())


        # results['loss'].append(loss)
        # results['accuracy'].append(acc)
    average_loss = sum(loss_list)/len(loss_list)
    average_acc = sum(acc_list)/(BATCH_SIZE*len(acc_list))*100
    #print(f"Current_val_loss: {loss}, val_accuracy:{acc}")
    return average_loss,average_acc



cuda:0


# Model to Compare

##Simplenet

In [ ]:
class simplenet(nn.Module):
    def __init__(self):
      super(simplenet,self).__init__()
      self.classifier = nn.Linear(256,100)

      self.avgpool = nn.AdaptiveAvgPool2d((1,1))

      self.block = nn.Sequential(
          #conv1
          nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(64, eps=1e-05, momentum=0.05, affine=True),
          nn.ReLU(),
          #conv2/3/4
          nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(128, eps=1e-05, momentum=0.05, affine=True),
          nn.ReLU(),
          nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(128, eps=1e-05, momentum=0.05, affine=True),
          nn.ReLU(),
          nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(128, eps=1e-05, momentum=0.05, affine=True),
          nn.ReLU(),
          #Maxpool
          nn.MaxPool2d(kernel_size = 2, stride = 2),
          #conv5/6
          nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(128, eps=1e-05, momentum=0.05, affine=True),
          nn.ReLU(),
          nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(128, eps=1e-05, momentum=0.05, affine=True),
          nn.ReLU(),
          #conv7
          nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(256, eps=1e-05, momentum=0.05, affine=True),
          nn.ReLU(),
          #Maxpool
          nn.MaxPool2d(kernel_size = 2, stride = 2),
          #conv8/9
          nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(256, eps=1e-05, momentum=0.05, affine=True),
          nn.ReLU(),
          nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(256, eps=1e-05, momentum=0.05, affine=True),
          nn.ReLU(),
          #Maxpool
          nn.MaxPool2d(kernel_size = 2, stride = 2),

          #conv10
          nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(512, eps=1e-05, momentum=0.05, affine=True),
          nn.ReLU(),

          #conv11
          nn.Conv2d(512, 2048, kernel_size=1, stride=1, padding=0),
          nn.BatchNorm2d(2048, eps=1e-05, momentum=0.05, affine=True),
          nn.ReLU(),

          #conv12
          nn.Conv2d(2048, 256, kernel_size=1, stride=1, padding=0),
          nn.BatchNorm2d(256, eps=1e-05, momentum=0.05, affine=True),
          nn.ReLU(),

          #Maxpool
          nn.MaxPool2d(kernel_size = 2, stride = 2),

          #conv13
          nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          )
      # for m in self.modules():
      #       if isinstance(m, nn.Conv2d):
      #           nn.init.xavier_uniform_(m.weight.data, gain=nn.init.calculate_gain("relu"))
    def forward(self, x):
        out = self.block(x)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

##New Model

In [ ]:
class newmodel(nn.Module):
    def __init__(self):
      super(newmodel,self).__init__()
      self.classifier = nn.Linear(432,100)

      self.avgpool = nn.AdaptiveAvgPool2d((1,1))

      self.block = nn.Sequential(
          nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(128),
          nn.ReLU(),
          nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(128),
          nn.ReLU(),
          nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(128),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size = 2, stride = 2),
          nn.Conv2d(128, 192, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(192),
          nn.ReLU(),
          nn.Conv2d(192, 192, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(192),
          nn.ReLU(),
          nn.Conv2d(192, 192, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(192),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size = 2, stride = 2),
          nn.Conv2d(192, 288, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(288),
          nn.ReLU(),
          nn.Dropout(0.3),
          nn.Conv2d(288, 288, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(288),
          nn.ReLU(),
          nn.Dropout(0.3),
          nn.Conv2d(288, 432, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(432),
          nn.ReLU(),
          nn.Dropout(0.3),
          nn.MaxPool2d(kernel_size = 2, stride = 2),
          )
    def forward(self, x):
        out = self.block(x)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out


## SqueezeNet1.0 and 1.1

In [ ]:
class Fire(nn.Module):
    def __init__(self, inplanes: int, squeeze_planes: int, expand1x1_planes: int, expand3x3_planes: int) -> None:
        super(Fire,self).__init__()
        self.inplanes = inplanes
        self.squeeze = nn.Conv2d(inplanes, squeeze_planes, kernel_size=1)
        self.squeeze_activation = nn.ReLU(inplace=True)
        self.expand1x1 = nn.Conv2d(squeeze_planes, expand1x1_planes, kernel_size=1)
        self.expand1x1_activation = nn.ReLU(inplace=True)
        self.expand3x3 = nn.Conv2d(squeeze_planes, expand3x3_planes, kernel_size=3, padding=1)
        self.expand3x3_activation = nn.ReLU(inplace=True)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.squeeze_activation(self.squeeze(x))
        return torch.cat(
            [self.expand1x1_activation(self.expand1x1(x)), self.expand3x3_activation(self.expand3x3(x))], 1
        )

class SqueezeNet(nn.Module):
    def __init__(self, num_classes = 100, dropout= 0.5) -> None:
        super(SqueezeNet, self).__init__()
        self.num_classes = num_classes
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=7, stride=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
            Fire(96, 16, 64, 64),
            Fire(128, 16, 64, 64),
            Fire(128, 32, 128, 128),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
            Fire(256, 32, 128, 128),
            Fire(256, 48, 192, 192),
            Fire(384, 48, 192, 192),
            Fire(384, 64, 256, 256),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
            Fire(512, 64, 256, 256),
        )

        # Final convolution is initialized differently from the rest
        final_conv = nn.Conv2d(512, self.num_classes, kernel_size=1)
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout), final_conv, nn.ReLU(inplace=True), nn.AdaptiveAvgPool2d((1, 1))
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if m is final_conv:
                    nn.init.normal_(m.weight, mean=0.0, std=0.01)
                else:
                    nn.init.kaiming_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.classifier(x)
        return torch.flatten(x, 1)

class SqueezeNet1_1(nn.Module):
  def __init__(self, num_classes = 100, dropout= 0.5) -> None:
      super(SqueezeNet1_1, self).__init__()
      self.num_classes = num_classes
      self.features = nn.Sequential(
              nn.Conv2d(3, 64, kernel_size=3, stride=2),
              nn.ReLU(inplace=True),
              nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
              Fire(64, 16, 64, 64),
              Fire(128, 16, 64, 64),
              nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
              Fire(128, 32, 128, 128),
              Fire(256, 32, 128, 128),
              nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
              Fire(256, 48, 192, 192),
              Fire(384, 48, 192, 192),
              Fire(384, 64, 256, 256),
              Fire(512, 64, 256, 256),
          )

      # Final convolution is initialized differently from the rest
      final_conv = nn.Conv2d(512, self.num_classes, kernel_size=1)
      self.classifier = nn.Sequential(
          nn.Dropout(p=dropout), final_conv, nn.ReLU(inplace=True), nn.AdaptiveAvgPool2d((1, 1))
      )

      for m in self.modules():
          if isinstance(m, nn.Conv2d):
              if m is final_conv:
                  nn.init.normal_(m.weight, mean=0.0, std=0.01)
              else:
                  nn.init.kaiming_uniform_(m.weight)
              if m.bias is not None:
                  nn.init.constant_(m.bias, 0)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
      x = self.features(x)
      x = self.classifier(x)
      return torch.flatten(x, 1)

## GoogLeNet

In [ ]:
class Inception(nn.Module):
    def __init__(self, input_channels, n1x1, n3x3_reduce, n3x3, n5x5_reduce, n5x5, pool_proj):
        super().__init__()


        self.b1 = nn.Sequential(
            nn.Conv2d(input_channels, n1x1, kernel_size=1),
            nn.BatchNorm2d(n1x1),
            nn.ReLU(inplace=True)
        )


        self.b2 = nn.Sequential(
            nn.Conv2d(input_channels, n3x3_reduce, kernel_size=1),
            nn.BatchNorm2d(n3x3_reduce),
            nn.ReLU(inplace=True),
            nn.Conv2d(n3x3_reduce, n3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(n3x3),
            nn.ReLU(inplace=True)
        )

        self.b3 = nn.Sequential(
            nn.Conv2d(input_channels, n5x5_reduce, kernel_size=1),
            nn.BatchNorm2d(n5x5_reduce),
            nn.ReLU(inplace=True),
            nn.Conv2d(n5x5_reduce, n5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n5x5, n5x5),
            nn.ReLU(inplace=True),
            nn.Conv2d(n5x5, n5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(inplace=True)
        )


        self.b4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(input_channels, pool_proj, kernel_size=1),
            nn.BatchNorm2d(pool_proj),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return torch.cat([self.b1(x), self.b2(x), self.b3(x), self.b4(x)], dim=1)

class GoogleNet(nn.Module):

    def __init__(self, num_class=100):
        super().__init__()
        self.prelayer = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 192, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
        )


        self.a3 = Inception(192, 64, 96, 128, 16, 32, 32)
        self.b3 = Inception(256, 128, 128, 192, 32, 96, 64)


        self.maxpool = nn.MaxPool2d(3, stride=2, padding=1)

        self.a4 = Inception(480, 192, 96, 208, 16, 48, 64)
        self.b4 = Inception(512, 160, 112, 224, 24, 64, 64)
        self.c4 = Inception(512, 128, 128, 256, 24, 64, 64)
        self.d4 = Inception(512, 112, 144, 288, 32, 64, 64)
        self.e4 = Inception(528, 256, 160, 320, 32, 128, 128)

        self.a5 = Inception(832, 256, 160, 320, 32, 128, 128)
        self.b5 = Inception(832, 384, 192, 384, 48, 128, 128)

        #input feature size: 8*8*1024
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout2d(p=0.4)
        self.linear = nn.Linear(1024, num_class)

    def forward(self, x):
        x = self.prelayer(x)
        x = self.maxpool(x)
        x = self.a3(x)
        x = self.b3(x)

        x = self.maxpool(x)

        x = self.a4(x)
        x = self.b4(x)
        x = self.c4(x)
        x = self.d4(x)
        x = self.e4(x)

        x = self.maxpool(x)

        x = self.a5(x)
        x = self.b5(x)

        x = self.avgpool(x)
        x = self.dropout(x)
        x = x.view(x.size()[0], -1)
        x = self.linear(x)

        return x

#AlexNet

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes = 100, dropout = 0.5) -> None:
        super(AlexNet,self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

## Resnet

In [ ]:
#Download from Pytorch

## MobileNet

In [ ]:
#Download from Pytorch

#Load model

In [ ]:
#My Model
mymodel = newmodel()
mymodel.load_state_dict(torch.load('/content/drive/MyDrive/DeepLearning/Assignment 2/Models selected/Models/mynewmodel_20epochs.pt'))
mymodel.to(device)

#SimpleNet
simple = simplenet()
simple.load_state_dict(torch.load('/content/drive/MyDrive/DeepLearning/Assignment 2/Models selected/Models/simplenet_homemade_20epochs.pt'))
simple.to(device)

#SqueezeNet
squeeze1 = SqueezeNet()
squeeze1.load_state_dict(torch.load('/content/drive/MyDrive/DeepLearning/Assignment 2/Models selected/Models/squeezeNet_20epochs.pt'))
squeeze1.to(device)

squeeze1_1 = SqueezeNet1_1()
squeeze1_1.load_state_dict(torch.load('/content/drive/MyDrive/DeepLearning/Assignment 2/Models selected/Models/squeezeNet1_1_20epochs.pt'))
squeeze1_1.to(device)

#GoogLeNet
ggnet = GoogleNet()
ggnet.load_state_dict(torch.load('/content/drive/MyDrive/DeepLearning/Assignment 2/Models selected/Models/googlenet_20epochs.pt'))
ggnet.to(device)

#ResNet18
resnet = resnet18(weights=None)
resnet.fc = nn.Linear(512, 100)
resnet.load_state_dict(torch.load('/content/drive/MyDrive/DeepLearning/Assignment 2/Models selected/Models/resnet18v1_20epochs.pt'))
resnet.to(device)

#MobileNetv2
mobile = mobilenet_v2(weights = None)
mobile.classifier[1] = nn.Linear(1280, 100)
mobile.load_state_dict(torch.load('/content/drive/MyDrive/DeepLearning/Assignment 2/Models selected/Models/mobilenetv2_20epochs.pt'))
mobile.to(device)

#AlexNet
alexnet = AlexNet()
alexnet.load_state_dict(torch.load('/content/drive/MyDrive/DeepLearning/Assignment 2/ModelList/AlexNet_20epochs'))
alexnet.to(device)



AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

#Number of Parameters

In [ ]:
modelList = [mymodel,simple,squeeze1,squeeze1_1,ggnet,alexnet,resnet,mobile]

for model in modelList:
    print(sum(p.numel() for p in model.parameters()))


3668404
5520996
786724
773796
6402564
57413540
11227812
2351972


#Inference time + accuracy

Skip first run (for warming up)

In [ ]:
for model in modelList:

    if model == ggnet or model == simple or model == mymodel:
        start = time.time()
        _,result = valid_model(model, testloader, device)
        print(model.__class__.__name__, '-', time.time() - start, "-accuracy:" ,result)
    else:
        start = time.time()
        _,result = valid_model(model, testloader_224, device)
        print(model.__class__.__name__, '-', time.time() - start, "-accuracy:" ,result)



newmodel - 4.2842512130737305 -accuracy: 68.08171400099651
simplenet - 3.246736526489258 -accuracy: 64.21524663677131
SqueezeNet - 17.18877410888672 -accuracy: 52.34678624813154
SqueezeNet1_1 - 18.397695302963257 -accuracy: 47.55356253114101
GoogleNet - 5.837626695632935 -accuracy: 65.9691081215745
AlexNet - 16.45577621459961 -accuracy: 50.14449427005481
ResNet - 16.961264610290527 -accuracy: 62.96960637767812
MobileNetV2 - 19.71480917930603 -accuracy: 62.610861983059294
